<a href="https://colab.research.google.com/github/Facusc16/data_flow/blob/main/Data_%26_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requerimientos

## Librerías

In [1]:
!pip install spotipy
import pandas as pd

from IPython.display import clear_output
import logging
import sys
import re
import json
import time
from datetime import timedelta

from google.colab import drive
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
from requests.exceptions import RequestException
import warnings

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 8.6 MB/s eta 0:00:00


## Google Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


## Construir DataFrame

In [3]:
path = '/content/drive/MyDrive/JSON/spotify_dataset.csv'

df = pd.read_csv(path)

# Analisis descriptivo

## Dimensiones del DataFreme

In [4]:
df.shape

(498052, 39)

## Información general del DataFrame

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498052 entries, 0 to 498051
Data columns (total 39 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Artist(s)                       498052 non-null  object 
 1   song                            498038 non-null  object 
 2   text                            498052 non-null  object 
 3   Length                          498052 non-null  object 
 4   emotion                         498052 non-null  object 
 5   Genre                           498052 non-null  object 
 6   Album                           498003 non-null  object 
 7   Release Date                    350369 non-null  object 
 8   Key                             498052 non-null  object 
 9   Tempo                           498052 non-null  float64
 10  Loudness (db)                   498052 non-null  float64
 11  Time signature                  498044 non-null  object 
 12  Explicit        

# Limpieza y manipulación de los datos

## Quitar datos duplicados o irrelevantes

In [6]:
df.duplicated().any()

np.False_

## Arreglar errores estructurales

In [7]:
df.head()

,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Good for Morning Routine,Similar Artist 1,Similar Song 1,Similarity Score 1,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,sadness,hip hop,Thr!!!er,2013-04-29,D min,0.437870,...,0,Corey Smith,If I Could Do It Again,0.986061,Toby Keith,Drinks After Work,0.983719,Space,Neighbourhood,0.983236
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,sadness,hip hop,Thr!!!er,2013-04-29,A# min,0.508876,...,0,Hiroyuki Sawano,BRE@TH//LESS,0.995409,When In Rome,Heaven Knows,0.990905,Justice Crew,Everybody,0.984483
2,!!!,Pardon My Freedom,"Oh my god, did I just say that out loud? Shoul...",05:51,joy,hip hop,Louden Up Now,2004-06-08,A Maj,0.532544,...,0,Ricky Dillard,More Abundantly Medley Live,0.993176,Juliet,Avalon,0.965147,The Jacksons,Lovely One,0.956752
3,!!!,Ooo,[Verse 1] Remember when I called you on the te...,03:44,joy,hip hop,As If,2015-10-16,A min,0.538462,...,0,Eric Clapton,Man Overboard,0.992749,Roxette,Don't Believe In Accidents,0.991494,Tiwa Savage,My Darlin,0.990381
4,!!!,Freedom 15,[Verse 1] Calling me like I got something to s...,06:00,joy,hip hop,As If,2015-10-16,F min,0.544379,...,0,Cibo Matto,Lint Of Love,0.981610,Barrington Levy,Better Than Gold,0.981524,Freestyle,Its Automatic,0.981415


In [8]:
df.tail()

,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Good for Morning Routine,Similar Artist 1,Similar Song 1,Similarity Score 1,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3
498047,박혜진 Park Hye Jin,Can you,Can you be my (Can you be my) Can you be my (C...,03:39,joy,house,Can you (Galcher Lustwerk Remix),2020-10-14,B min,0.644970,...,0,LXST CXNTURY,NEVER EXISTED,0.946429,"Occvlt,StoneDogg",ALIZAY,0.926155,Peaches,Light in Places,0.923749
498048,심형진 Hyungjin Sim,Lim Han Byul Lee Hyuk - Prayer English Transl...,[Verse] I gave you my everything But I see I c...,08:47,joy,metalcore,비 준비하시니 Psalm_147,2022-04-19,G# Maj,0.573964,...,0,Kari Jobe,No Fear Live,0.991961,"The Belonging Co,Kari Jobe",Closer To Your Heart,0.990956,Saintseneca,Pillar of Na,0.990385
498049,우정하,Memories Smiling Tears Remix,"[Tasha] Hell on earth, watching ...",03:53,sadness,hip hop,MEMORY,2023-08-30,G# min,0.260355,...,0,Manic Street Preachers,People Give In,0.982809,Vinyl Theatre,Gold,0.980835,Unspoken,Life in the Death of Me,0.980171
498050,제노,Come Back to Me,[Verse] Gifts and flowers lay upon the lid Of ...,03:12,sadness,hip hop,2008 to me again,NaN,E Maj,0.562130,...,0,Bonnie Raitt,I Can't Help Myself,0.983805,J. Roddy Walston & The Business,Bad Habits,0.982425,Idina Menzel,Damsel In Distress,0.975935
498051,코케,I Wanna Be With You,I wanna b with you I wanna b with you baby Yea...,03:32,love,hip hop,I Wanna Be With You : Originally Performed By ...,2020-11-04,D Maj,0.408284,...,0,Ameritz - Karaoke,Roxanne 97 Puff Daddy Remix,0.987132,Ameritz - Karaoke,Roxanne,0.987132,ProSource Karaoke,Get Back Single Version,0.984674


In [9]:
df.sample(5)

,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Good for Morning Routine,Similar Artist 1,Similar Song 1,Similarity Score 1,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3
140181,Exposé,Now That I Found You,I've been searching (for so long) For the love...,03:46,joy,pop,What You Don't Know (Expanded Edition),NaN,D min,0.520710,...,0,Harry Belafonte,Amandla,0.985183,Boney M.,Lady Godiva,0.984351,Alex Lahey,L-L-L-Leave Me Alone,0.983481
186112,Ian Isiah,Bedroom,[Intro] My bedroom My bedroom [Verse 1] Got tw...,02:43,love,hip hop,Shugga Sextape (Vol. 1),NaN,E min,0.810651,...,0,Aswad,Need Your Love Each And Every Day,0.976429,"Heidi Tiffany,ZJ Dymond",Call Me,0.975250,"Charles Strouse,Chita Rivera,Susan Watson,Orig...",What Did I Ever See In Him?,0.973432
206542,Jesus Loves You,Bow Down Mister (A Small Portion 2B Polite Mix),From Bombay to Bangalore All the Hindus kno...,03:45,joy,house,At Worst...The Best Of Boy George And Culture ...,NaN,G Maj,0.437870,...,0,Jesus Loves You,Bow Down Mister,0.998995,Jazmine Sullivan,Dream Big,0.988045,"Anevo,Jae-Mi",Dont Shoot Me Down,0.987947
316924,Octayvian,Untitled,[Shizzy] Bitches give me play cause I'm the ha...,01:36,joy,hip hop,FROM TH3 H3ART WITH LOV3,2023-12-29,C# min,0.544379,...,0,TesseracT,Orbital,0.988792,Michael Giacchino,Welcome Back,0.988573,"Christmas All Year Round,XMAS Moods 2020,Holly...",A Boy Is Born In Bethlehem Christmas,0.986970
149557,Forth Wanderers,Fuck,You know I was thinking a lot that day but I s...,02:27,joy,hip hop,Tough Love,2024-07-26,C Maj,0.585799,...,0,The Lawrence Arms,Quiet Storm,0.969286,RMB,Dayman,0.966484,Babyshambles,Pipedown,0.965765


### Definir errores

#### Cambiar nombres de columnas a minúsculas, y reemplazar espacios y carácteres especiales

In [10]:
trans = str.maketrans({
    " ": "_",
    "/": "_",
    "(": "",
    ")": ""
})

columns = {}
for column in df.columns:
  columns[column] = column.strip().lower().translate(trans)

df.rename(columns=columns, inplace=True)

#### Separar "Artists" en distintas filas

In [11]:
# Encontré esta fila posteriormente, "Clifton Davins," estaba creando una fila más al hacer .split(), .explode() por tener una coma de más.
df.loc[469650, 'artists'] = 'Vanessa Bell Armstrong, Sandra Crouch, Patti Austin, Daryl Coley, Commissioned, Clifton Davis, Bernie K., Andrae Crouch'

In [12]:
df['artists'] = df['artists'].str.split(r',|;')

In [13]:
df = df.explode('artists').reset_index(drop=True)

#### Separar "genre" en distintas filas

In [14]:
df['genre'] = df['genre'].str.split(",")

In [15]:
df = df.explode('genre').reset_index(drop=True)

#### Eliminar espacios extras de columnas tipo object

In [16]:
for col in df.select_dtypes(include='object'):
  if len(df[df[col].notna() & (df[col].str.strip() != df[col])]) != 0:
    df[col] = df[col].str.strip()

#### Reemplazar strings vacios en 'artists'

In [17]:
artists = ['Chhrmng,', 'Desconocido', 'Desconocido', 'Desconocido']
for i, index in enumerate(df[df['artists'].apply(lambda x: x.strip() == '')].index):
  df.loc[index, 'artists'] = artists[i]

#### Convertir a minúscula "text" y "emotion"

In [18]:
df['text'] = df['text'].str.lower()

In [19]:
df['emotion'] = df['emotion'].str.lower()

#### Convertir "length" a timedelta

In [20]:
df['length'] = df['length'].apply(lambda x: timedelta(minutes=int(x.split(':')[0]), seconds=int(x.split(':')[1])))

#### Cambiar "release_date" a datetime

In [21]:
df['release_date'] = pd.to_datetime(df['release_date'])

#### Cambiar valores de "explicit" a tipo bool

In [22]:
df['explicit'] = df['explicit'].map({'Yes': True, 'No': False})

#### Cambiar todos los valores binarios de todos los "Good for" a tipo bool

In [23]:
for column in df.columns[21:30]:
  df[column] = df[column].map({1: True, 0: False})

In [24]:
df = df.drop_duplicates()

## **Filtrar outliers**

In [25]:
desc = df.describe().loc[:, ['length', 'tempo', 'loudness_db']]
desc

,length,tempo,loudness_db
count,1005026,1.005026e+06,1.005026e+06
mean,0 days 00:03:49.560608382,5.321303e-01,7.624414e-01
min,0 days 00:00:05,0.000000e+00,0.000000e+00
25%,0 days 00:03:02,3.905325e-01,7.287427e-01
50%,0 days 00:03:38,5.266272e-01,7.785247e-01
75%,0 days 00:04:20,6.449704e-01,8.123183e-01
max,0 days 00:59:32,1.000000e+00,1.000000e+00
std,0 days 00:01:32.628351743,1.726622e-01,7.320502e-02


In [26]:
bounds = {}

for column in desc.columns:
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)

  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  if df[column].min() < lower_bound or df[column].max() > upper_bound:
    bounds[column] = lower_bound, upper_bound

bounds

{'length': (Timedelta('0 days 00:01:05'), Timedelta('0 days 00:06:17')),
 'tempo': (np.float64(0.008875739700000052), np.float64(1.0266272188999999)),
 'loudness_db': (np.float64(0.6033793605), np.float64(0.9376816860999999))}

In [27]:
len(df[
    (df['length'] <= bounds['length'][0]) |
    (df['length'] >= bounds['length'][1]) |
    (df['loudness_db'] <= bounds['loudness_db'][0]) |
    (df['loudness_db'] >= bounds['loudness_db'][1])
])

80949

Voy a dividir el estudio entre analisis con outliers y analisis sin outliers, ya que en base al análisis de los mismos determiné que probablemente sean valores reales en su gran mayoría y no algún tipo de error de lectura o escritura.

Por otro lado, las columnas que presentan outliers no deberían interferír en las respuestas a las principales preguntas a contestar en este Análisis Exploratorio de los Datos, ya que estás apelan a la información en otras columnas.

El doble análisis será, entonces, con el fin de desarrollar un modelo de machine learning de clasificación y comparar los resultados para el mismo modelo con y sin valores atípicos.

In [28]:
df_clipped = df.copy()

for column, (lower, upper) in bounds.items():
  df_clipped[column] = df_clipped[column].clip(lower=lower, upper=upper)

## Lidiar con NaNs

In [29]:
df.isna().sum()[df.isna().sum() > 0]

,0
song,41
album,99
release_date,305092
time_signature,13
similar_song_1,35
similar_song_2,23
similar_song_3,6


### song / similar_song

In [30]:
def fill_values(name, song, columns):
  for col_artist, col_song in columns:
    rows = (df[col_artist].apply(lambda x: any(a in name for a in x.split(','))) & (df[col_song].isna()))
    df.loc[rows, col_song] = song

In [31]:
to_fill = {
    'Dylan Frost': {
        'name': "Dylan Frost",
        'song': "Velvet Diamonds",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2'), ('similar_artist_3', 'similar_song_3')]},

    'Yxngxr1': {
        'name': "Yxngxr1",
        'song': "Desconocido",
        'columns': [('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2'), ('similar_artist_3', 'similar_song_3')]},

    'HOMESHAKE': {
        'name': "HOMESHAKE",
        'song': "Believe",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2')]},

    'RAP CON CLAVE,El Nano': {
        'name': "RAP CON CLAVE,El Nano",
        'song': "Oshun con Clave",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2')]},

    'RATKING,King Krule': {
        'name': "RATKING,King Krule",
        'song': "So Sick Stories",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2')]},

    'Skillibang,Aidonia,Najeeriii,6t6': {
        'name': "Skillibang,Aidonia,Najeeriii,6t6",
        'song': "Desconocido",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2')]},

    "Stan Walker": {
        'name': 'Stan Walker',
        'song': '"He Kakano Ahau - Live"',
        'columns': [('artists', 'song'), ('similar_artist_2', 'similar_song_2'), ('similar_artist_3', 'similar_song_3')]},

    "Charles Mingus": {
        'name': "Charles Mingus",
        'song': "Moanin'",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2')]},

    "Miles Davis": {
        'name': "Miles Davis",
        'song': 'Santana',
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1'), ('similar_artist_2', 'similar_song_2')]},

    'john_eliot': {
        'name': "Christoph Willibald Gluck,Thomas Allen,John Aler,Nancy Argenta,Sophie Boulin,Lucinda Houghton,Jane Armstrong,Suzanne Flowers,Carol Hall,Nicola Jenkin,Jean Knibbs,Rachel Platt,Mary Seers,The Monteverdi Choir,Orchestre de l'Opéra de Lyon,John Eliot Gardiner",
        'song': "Iphigénie en Tauride / Act 4: Scène 7. Récitatif. 'Ta soeur! qu'ai-je entendu?' - Air. 'Dans cet objet touchant' - Choeur final. 'Les Dieux, longtemps en courroux'",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1')]},

    'Tally Hall': {
        'name': "Tally Hall",
        'song': "Welcome to Tally Hall",
        'columns': [('artists', 'song'), ('similar_artist_2', 'similar_song_2')]},

    'Pinegrove': {
        'name': "Pinegrove",
        'song': "Need 2",
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1')]},

    "JAZZY NANU": {
        'name': 'JAZZY NANU',
        'song': 'BILLIE EILISH',
        'columns': [('artists', 'song'), ('similar_artist_2', 'similar_song_2')]},

    "Gene Watson": {
        'name': 'Gene Watson',
        'song': 'Fourteen Carat Mind',
        'columns': [('artists', 'song'), ('similar_artist_1', 'similar_song_1')]},

    "Ramandeep Bawa's/Nazma Khan's": {
        'name': "Ramandeep Bawa's,Nazma Khan's",
        'song': 'Desconocido',
        'columns': [('artists', 'song')]}
}

In [32]:
for key, value in to_fill.items():
  name = value.get('name').split(',')
  song = value.get('song')
  columns = value.get('columns')
  fill_values(name, song, columns)

### album

In [33]:
to_fill_albums = {
    'destroy_lonely': {
        'name': "Destroy Lonely",
        'album': "ᐸ/3²",
        'columns': [('artists', 'album')]
    },
    'nayeon': {
        'name': "NAYEON",
        'album': "Desconocido",
        'columns': [('artists', 'album')]
    },
    'meshuggah': {
        'name': "Meshuggah",
        'album': "None",
        'columns': [('artists', 'album')]
    },
    'artms': {
        'name': "ARTMS",
        'album': "Desconocido",
        'columns': [('artists', 'album')]
    },
    'jfth': {
        'name': "Jfth",
        'album': "<|°_°|>",
        'columns': [('artists', 'album')]
    },
    'Gupi,Fraxiom,food house': {
        'name': "Gupi,Fraxiom,food house",
        'album': "None",
        'columns': [('artists', 'album')]
    }
}

In [34]:
for key, value in to_fill_albums.items():
  name = value.get('name').split(',')
  song = value.get('album')
  columns = value.get('columns')
  fill_values(name, song, columns)

In [35]:
# Filtrar filas
caravan_palace = (df['artists'] == 'Caravan Palace') & (df['album'].isna()) & (df['song'] != 'Im In Trouble')

# Asignar valores
df.loc[caravan_palace, 'album'] = '<|°_°|>'
df.loc[df['song'] == 'Im In Trouble', 'album'] = 'Desconocido'

In [36]:
# Asignar valores
df.loc[(df['artists'] == "WILLOW") & (df['song'] == 'Maybe'), 'album'] = 'Coping Mechanism'
df.loc[(df['artists'] == "WILLOW") & (df['song'] == 'IM NOT OKAY I PROMISE'), 'album'] = 'Desconocido'

### release_date

La estrategía para reemplazar los valores NaT en la columna 'release_date' va a ser asignar las fechas mediante la columna 'album', los cuales contendran filas que, al pertenecer a un album, tendrán todas la misma fecha.

La agrupación de filas por 'artists' y 'album' aumenta considerablemente la cantidad de grupos, pero será necesaria para diferenciar aquellos albums que comparten nombres pero que no son el mismo album.

#### Completar con valores existentes en el DF

In [37]:
# Creo un DF con las filas donde 'release_date' no sea NaN
release_dates = df[df['release_date'].notna()]

# Agrupar DataFrame por 'artists' y 'album' y acceder a la columna 'release_date'
release_dates = release_dates.groupby(['artists', 'album'])['release_date']

# Me quedo con el primer valor de cada grupo para 'release_date'
release_dates = release_dates.first()

# Transformo el Series a un DataFrame
release_dates = release_dates.reset_index()

In [38]:
# Obtener, para todas las filas donde 'release_date' sea NaN, coincidencias entre ('artists', 'album') con el DataFrame con los valores faltantes para cada caso
matches = df[df['release_date'].isna()].merge(release_dates, on=['artists', 'album'], how='left', suffixes=('', '_filled'))

# Reemplazar 'release_date' por 'release_date_filled'
matches['release_date'] = matches['release_date_filled']

# Eliminar 'realease_date_filled'
matches = matches.drop(columns='release_date_filled')

In [39]:
df.loc[df['release_date'].isna(), 'release_date'] = matches['release_date']

#### Completar con valores obtenidos de la API de Spotify

##### Crear excepción para detener la ejecución del programa de forma limpia al alcanzar el límite de peticiones a la API de Spotify

In [40]:
# Crear exception personalizada
class RateLimitReached(Exception):
  pass

# Crear Handler personalizado
class RateLimitHandler(logging.Handler):

  # Método para procesar y manejar cada mensaje de log
  def emit(self, record):

    # Obtener mensaje del log
    msg = record.getMessage()

    # Verificar si el mensaje es el que buscamos
    if "your application has reached a rate/request limit" in msg.lower():

      # Establecer mensaje a mostrar y detener la ejecución del código
      match = re.search(r"after: (\d+)", msg)
      if match:
        retry_seconds = int(match.group(1))
        hours = retry_seconds // 3600
        minutes = (retry_seconds % 3600) // 60
        raise RateLimitReached(f"Se alcanzó el límite de peticiones. Volver a ejecutar después de {hours}h. {minutes}min.")

# Obtener logger principal
logger = logging.getLogger()

# Evitar que se imprima el WARNING original
logger.propagate = False

# Agregar handler personalizado
logger.addHandler(RateLimitHandler())

# Configurar solo para mensjes tipo WARNING o superior
logger.setLevel(logging.WARNING)

##### Crear función para cargar/crear diccionario con fechas faltantes

In [41]:
def get_album_release_date(df, client_id, client_secret, release_dates_data, empty_dates, cache_file='/content/drive/MyDrive/JSON/release_dates_cache.json'):

  def get_album_date(artist, album, retries=3, delay=1):

    for attempt in range(retries):
      try:

        query = f"album:{album} artist:{artist}"
        results = sp.search(query, type='album', limit=1)

        items = results['albums']['items']
        if items:
          return items[0]['release_date']
        return None

      except (RequestException, SpotifyException) as e:

        if isinstance(e, SpotifyException) and e.http_status == 429:

          retry_after = int(e.headers.get("Retry-After", 0))
          horas = retry_after // 3600
          minutos = (retry_after % 3600) // 60
          clear_output(wait=True)
          raise RateLimitReached(f"Se alcanzó el límite de peticiones. Volver a ejecutar después de {hours}h. {minutes}min.")


        else:
          clear_output(wait=True)
          print(f"Error in the request ({e}). Retrying ({attempt+1}/{retries})...")
          time.sleep(delay)

    return None

  auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
  sp = spotipy.Spotify(auth_manager=auth_manager)

  count = 0
  for i, row in empty_dates.iterrows():
    key = f"{row['artists']}||{row['album']}"

    if key not in release_dates_data:
      count += 1
      release_dates_data[key] = get_album_date(row['artists'], row['album'])

      with open(cache_file, 'w', encoding='utf-8')as f:
        json.dump(release_dates_data, f, ensure_ascii=False, indent=2)

      time.sleep(0.1)

    if count % 1000 == 0 and count != 0:
      clear_output(wait=True)
      print(f"Procesados {count} albumes...")

  return release_dates_data

##### Cargar/crear/completar diccionario con fechas faltantes

In [42]:
client_id = ''
client_secret = ''
empty_dates = df[df['release_date'].isna()][['artists', 'album']].drop_duplicates()

try:
  with open('/content/drive/MyDrive/JSON/release_dates_cache.json', 'r', encoding='utf-8') as f:
     release_dates_data = json.load(f)

except FileNotFoundError:
  release_dates_data = {}

try:
  if len(release_dates_data) != len(empty_dates):
    release_dates_data = get_album_release_date(df, client_id, client_secret, release_dates_data, empty_dates)
  else:
    print("El archivo ya ha sido completado")

except RateLimitReached as rlt:
  print(rlt)


print(f"Cargados: {len(release_dates_data)}/{len(empty_dates)} albumes")

El archivo ya ha sido completado
Cargados: 85407/85407 albumes


##### Crear DataFrame con fechas faltantes

In [43]:
rows = [(*key.split("||", maxsplit=1), value) for key, value in release_dates_data.items()]

release_dates_df = pd.DataFrame(rows, columns=['artists', 'album', 'release_date'])

##### Limpiar DataFrame con fechas faltantes

###### Corregir valores en el año 0000

In [44]:
año_cero = release_dates_df['release_date'] == '0000'
release_dates_df.loc[año_cero, 'release_date'] = None

###### Asignar valores de mes y día promedio a las fechas que solo cuentan con el año de publicación

In [45]:
years = release_dates_df[release_dates_df['release_date'].str.len() < 5].sort_values(by='release_date')['release_date'].unique()

for year in years:
  mean_date = df[df['release_date'].dt.year == int(year)]['release_date'].mean().to_pydatetime()

  release_dates_df.loc[release_dates_df['release_date'] == year, 'release_date'] = mean_date

/tmp/ipython-input-45-98120455.py:4: UserWarning: Discarding nonzero nanoseconds in conversion.
  mean_date = df[df['release_date'].dt.year == int(year)]['release_date'].mean().to_pydatetime()


###### Asignar valor por defecto a las fechas que solo cuentan con año y mes de publicación

In [46]:
year_month = release_dates_df['release_date'].str.len() < 8
release_dates_df.loc[year_month, 'release_date'] = release_dates_df.loc[year_month, 'release_date'] + '-01'

###### Transformar columna 'release_date' a tipo datetime, y normalizar horario a 00:00:00

In [47]:
release_dates_df['release_date'] = pd.to_datetime(release_dates_df['release_date'])
release_dates_df['release_date'] = release_dates_df['release_date'].dt.normalize()

##### Asignar nuevas fechas a los valores faltantes

In [48]:
# Obtener, para todas las filas donde 'release_date' sea NaN, coincidencias entre ('artists', 'album') con el DataFrame con los valores faltantes para cada caso.
matches = df[df['release_date'].isna()].merge(release_dates_df, on=['artists', 'album'], how='left', suffixes=('', '_filled'))

# Reemplazar 'release_date' por 'release_date_filled'
matches['release_date'] = matches['release_date_filled']

# Reemplazar los índices en matches, por los índices originales de las filas en df
matches.index = df[df['release_date'].isna()].index

# Eliminar 'realease_date_filled'
matches = matches.drop(columns='release_date_filled')

In [49]:
df.loc[matches.index, 'release_date'] = matches['release_date']

In [50]:
# Llenar las últimas filas NaT con una fecha equivalente a un valor "Desconocido"
df['release_date'] = df['release_date'].fillna(pd.Timestamp('1700-01-01'))

### time_signature

In [51]:
# Aislar géneros únicos de filas donde 'time_signature' es NaN
missing_time_signatures = df[df['time_signature'].isna()]['genre'].unique()

# A cada fila donde 'time_signature' es NaN, asignar para cada género el valor de time signature más frecuente
for genre in missing_time_signatures:
  df.loc[(df['time_signature'].isna()) & (df['genre'] == genre), 'time_signature'] = df[df['genre'] == genre]['time_signature'].value_counts().index[0]

## Validar nuestros datos

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1005026 entries, 0 to 1005087
Data columns (total 39 columns):
 #   Column                          Non-Null Count    Dtype          
---  ------                          --------------    -----          
 0   artists                         1005026 non-null  object         
 1   song                            1005026 non-null  object         
 2   text                            1005026 non-null  object         
 3   length                          1005026 non-null  timedelta64[ns]
 4   emotion                         1005026 non-null  object         
 5   genre                           1005026 non-null  object         
 6   album                           1005026 non-null  object         
 7   release_date                    1005026 non-null  datetime64[ns] 
 8   key                             1005026 non-null  object         
 9   tempo                           1005026 non-null  float64        
 10  loudness_db                     100

In [53]:
df.duplicated().any()

np.False_

In [54]:
df.isna().any()

,0
artists,False
song,False
text,False
length,False
emotion,False
genre,False
album,False
release_date,False
key,False
tempo,False


# EDA

## ¿Cual es el top 10 de las canciones que generan mejor flow?

## ¿Que artista es el que transmite más flow?

## Clasifica el top 3 de canciones según estas categorías:

### Fiesta

### Trabajo

### Ejercicio

### Relajación

### Conducción

## Visualizaciones:

### Cantidad de positivismo por año

### Cantidad de canciones por año y género

## Extra Challenge